In [1]:
QRELS_PATH = 'evaluation/qrels.json'

In [58]:
SEED = 2024
ANALYSIS_COL = 'userAcceleration.z'
PRECOMPUTED_DISTANCES = f"evaluation/adtw/{ANALYSIS_COL.replace('.', '-')}_distances.npy"
RUN_FILE = f"evaluation/runs/adtw_{ANALYSIS_COL.replace('.', '-')}.json"

# DIST = 'adtw'
# PRECOMPUTED_DISTANCES = f"evaluation/multivar/{DIST}_distances.npy"
# RUN_FILE = f"evaluation/runs/{DIST}_multivar.json"

# Preparando avaliação

In [59]:
from ranx import Qrels

In [60]:
qrels = Qrels.from_file(QRELS_PATH)

# Executando a busca por similaridade

In [61]:
from ranx import Run
import numpy as np
from aeon.distances import dtw_pairwise_distance

In [62]:
def compute_neighbors_by_similarity(
    distances: np.array,
    topk: int = 100) -> dict:

  run_dict = {}

  for qid in range(distances.shape[0]):
    all_sims = distances[qid]
    topk = np.argsort(all_sims)

    # ignorando o proprio exemplo de busca
    estimated_ids = topk[1:]
    scores = list(range(len(estimated_ids)))[::-1]

    run_dict[str(qid)] = {str(n): float(score) for n, score in zip(estimated_ids, scores)}

  return run_dict

In [63]:
PRECOMPUTED_DISTANCES

'evaluation/adtw/userAcceleration-z_distances.npy'

In [64]:
distances = np.load(PRECOMPUTED_DISTANCES)

In [65]:
run_dict = compute_neighbors_by_similarity(distances)

# Avaliando métricas de RI

In [66]:
from ranx import evaluate

In [67]:
METRICS = ["recall@10", 'map@10', 'hit_rate@10', 'mrr@10']
# METRICS = 'mrr@10'

In [68]:
run = Run(run_dict)

In [69]:
metrics = evaluate(qrels, run, METRICS)
metrics

{'recall@10': 0.21846064814814814,
 'map@10': 0.1649327371766608,
 'hit_rate@10': 0.9861111111111112,
 'mrr@10': 0.7505649250440916}

# Serializando Run

In [70]:
RUN_FILE

'evaluation/runs/adtw_userAcceleration-z.json'

In [71]:
run.save(RUN_FILE)